In [1]:
import numpy as np
import math
import cv2

font = cv2.FONT_HERSHEY_PLAIN
text_color = (255,255,255)

cap = cv2.VideoCapture(0)


top_left = (245,50)
bottom_right = (580,295)

while True:
    ret, frame = cap.read()

    frame = cv2.flip(frame, 1)

    h, w = frame.shape[:2] # h, w = 480, 640

    frame[0: 40, w-200: w] = (0) # (440,0) -> (640,40)

    cv2.putText(frame, "Fingers : ", (w-190, 30), font, 2, text_color, 2) 
    
    cv2.rectangle(frame, (top_left[0]-5, top_left[1]-5), (bottom_right[0]+5, bottom_right[1]+5), (0,255,255), 3)

    
    test_window = frame[top_left[1]: bottom_right[1], top_left[0]: bottom_right[0]]

    test_window_blurred = cv2.GaussianBlur(test_window, (5,5), 0)
    
    hsv = cv2.cvtColor(test_window_blurred, cv2.COLOR_BGR2HSV)

    lower_color = np.array([0, 24, 0])
    upper_color = np.array([179,255,255])

    mask = cv2.inRange(hsv, lower_color, upper_color)
    cv2.imshow("Mask", mask) 

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE) #cv2.RETR_EXTERNAL

    if len(contours) > 0:
       
        max_contour = max(contours, key = cv2.contourArea)
        cv2.drawContours(test_window, max_contour, -1, (255,0,0), 3)
        
        convhull = cv2.convexHull(max_contour, returnPoints = True) 
        cv2.drawContours(test_window, [convhull], -1, (0,0,255), 3, 2)

        min_y = h 
        final_point = (w, h)
        for i in range(len(convhull)):
            point = (convhull[i][0][0], convhull[i][0][1])
            if point[1] < min_y:
                min_y = point[1]
                final_point = point
        cv2.circle(test_window, final_point, 5, (0), 2)
    
        M = cv2.moments(max_contour) 
        if M["m00"]!=0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.circle(test_window, (cX, cY), 6, (0,0,255), 3)
            
        
        contour_poly = cv2.approxPolyDP(max_contour, 0.01*cv2.arcLength(max_contour,True), True)
        #cv2.fillPoly(test_window, [max_contour], text_color) 

        hull = cv2.convexHull(contour_poly, returnPoints = False) 

        defects = cv2.convexityDefects(contour_poly, hull)
        if defects is not None:
            count = 0
            points = []
            for i in range(defects.shape[0]): 
                start_index, end_index, far_pt_index, fix_dept = defects[i][0]
                start_pts = tuple(contour_poly[start_index][0])
                end_pts = tuple(contour_poly[end_index][0])
                mid_pts = (int((start_pts[0]+end_pts[0])/2), int((start_pts[1]+end_pts[1])/2))
                far_pts = tuple(contour_poly[far_pt_index][0])
                
                points.append(mid_pts)
# ! #                cv2.circle(test_window, mid_pts, 2, (0,255,255), 2) 
                
# ! #                 cv2.circle(test_window, start_pts, 2, (0,255,255), 2)
                
# ! #                 cv2.circle(test_window, end_pts, 2, (0), 2)
                
# ! #                 cv2.circle(test_window, far_pts, 2, text_color, 2)

                cv2.line(test_window, start_pts, end_pts, (0,255,0), 2)   
                cv2.line(test_window, start_pts, far_pts, (0,255,255), 2) 
                cv2.line(test_window, end_pts, far_pts, text_color, 2)   

                
                
                a = np.sqrt((end_pts[0] - start_pts[0])**2 + (end_pts[1] - start_pts[1])**2)
                b = math.sqrt((far_pts[0] - start_pts[0])**2 + (far_pts[1] - start_pts[1])**2)
                c = math.sqrt((end_pts[0] - far_pts[0])**2 + (end_pts[1] - far_pts[1])**2)
                
                angle = math.acos((b**2 + c**2 - a**2) / (2*b*c))  

                if angle <= np.pi/2:  
                    count += 1

                    frame[0:40, w-40:w] = (0)
                    for c in range(5):
                        if count == c:
                            cv2.putText(frame, str(count+1), (w-35,30), font, 2, text_color, 2)
                if len(points) <= 1 :
                    frame[0:40, w-40:w] = (0)
                    cv2.putText(frame, "1", (w-35,30), font, 2, text_color, 2)

  
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(25)
    if key==27: 
        break
    

cap.release()
cv2.destroyAllWindows()